In [67]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

In [71]:
def main():
    # 파일 불러오기
    movie_raw = pd.read_csv("movie_final.csv", dtype={'관객수':np.str_, '스크린수':np.str_, '상영횟수':np.str_})
    movie_raw = movie_raw.loc[:, "영화명": "배우"]
    movie = movie_raw

    # 데이터 추출 후 Nan을 빈값으로 바꿔줌
    movie = movie.fillna('')
    
    movie = movie[:6810]
    #movie = movei[6810:13620] 종혁
    #movie = movei[16320:20430] 승훈
    #movie = movei[20430:] 대우

    # 엑셀파일 만들기
    outWorkbook = xlsxwriter.Workbook("Distributor"+ datetime.today().strftime("%Y%m%d%H%M%S")+".xlsx")
    worksheet = outWorkbook.add_worksheet("Distributor")

    #엑셀의 해더를 만든다.
    excel_export_head(worksheet)


    # 엑셀 1행부터 시작하기 위해 선언
    cnt = 1

    # 예외처리 후 프로그램이 돌다 죽을 경우를 대비해서 엑셀파일 생성 여부를 판단하는 구분자.
    excel_yn = False

    # CSV 파일에서 읽은 데이터 프레임을 행별로 뽑는다.
    for index,row in movie.iterrows():

        distribution = row['배급사']
        grade = row['등급']
        genre = row['장르']
        actor = row['배우']
        date = row['개봉일']
        director = row['감독']
        nation = row['대표국적']

        # 배급사, 등급, 장르, 배우가 모두 있는경우 크롤링 안함.
        if(len(distribution) > 0 and len(grade) > 0  and len(genre) > 0 and len(actor) > 0 and len(date) > 0 and len(director)):
            movie_dic = {}
            movie_dic['배급사'] = distribution
            movie_dic['등급'] = grade
            movie_dic['장르'] = genre
            movie_dic['배우'] = actor
            movie_dic['개봉일'] = date
            movie_dic['감독'] = director
            movie_dic['대표국적'] = nation

            print("전처리데이터:", cnt, row['영화명'], " -> ", movie_dic)
            print("--------------------------------------------------")
            # 엑셀파일 생성
            excel_export(worksheet, cnt, row, movie_dic)

        else:
            
            #movie_dic = movie_search_list(row)
            # 영화리스트에서 해당영화를 찾고 데이터 딕셔너리를 리턴한다.
            try:
                movie_dic = movie_search_list(row)

            except Exception as e:
                print("예외처리된 영화명 :", cnt, row['영화명'])
                print('예외가 발생했습니다. 예외처리 내용', e)
                
                break

            # 받은 데이터 딕셔너리를 엑셀에 출력한다.
            print("크롤링데이터:", cnt, row['영화명'], " -> ", movie_dic)
            print("--------------------------------------------------")
            excel_export(worksheet, cnt, row, movie_dic)

        # 다음행을 가르키기 위해 하나씩 더한다.
        cnt += 1


    outWorkbook.close()

In [72]:
# 엑셀의 헤더
def excel_export_head(worksheet):

    worksheet.write(0, 0, "영화명")
    worksheet.write(0, 1, "개봉일")
    worksheet.write(0, 2, "매출액")
    worksheet.write(0, 3, "누적매출액")
    worksheet.write(0, 4, "관객수")
    worksheet.write(0, 5, "누적관객수")
    worksheet.write(0, 6, "스크린수")
    worksheet.write(0, 7, "상영횟수")
    worksheet.write(0, 8, "대표국적")
    worksheet.write(0, 9, "배급사")
    worksheet.write(0, 10, "등급")
    worksheet.write(0, 11, "장르")
    worksheet.write(0, 12, "감독")
    worksheet.write(0, 13, "배우")

In [73]:
# 엑셀 데이터 쓰기
def excel_export(worksheet,cnt, row,movie_dic):
    worksheet.write(cnt, 0, row['영화명'])
    
    if(row["개봉일"] == ''):
        worksheet.write(cnt, 1, movie_dic["개봉일"])
    else:
         worksheet.write(cnt, 1, row["개봉일"])
            
    worksheet.write(cnt, 2, row["매출액"])
    worksheet.write(cnt, 3, row["누적매출액"])
    worksheet.write(cnt, 4, row["관객수"])
    worksheet.write(cnt, 5, row["누적관객수"])
    worksheet.write(cnt, 6, row["스크린수"])
    worksheet.write(cnt, 7, row["상영횟수"])
    
    if(row["대표국적"] == ''):
        if(movie_dic["대표국적"] != ""):
            worksheet.write(cnt, 8, movie_dic["대표국적"])
        else:
            worksheet.write(cnt, 8, "")
    else:
         worksheet.write(cnt, 8, row["대표국적"])
    
    if(row["배급사"] == ''):
        worksheet.write(cnt, 9, movie_dic["배급사"])
    else:
         worksheet.write(cnt, 9, row["배급사"])
            
    if(row["등급"] == ''):
        worksheet.write(cnt, 10, movie_dic["등급"])
    else:
         worksheet.write(cnt, 10, row["등급"])
            
    if(row["장르"] == ''):
        worksheet.write(cnt, 11, movie_dic["장르"])
    else:
         worksheet.write(cnt, 11, row["장르"])
            
    if(row["감독"] == ''):
        worksheet.write(cnt, 12, movie_dic["감독"])
    else:
         worksheet.write(cnt, 12, row["감독"])
            
    if(row["배우"] == ''):
        worksheet.write(cnt, 13, movie_dic["배우"])
    else:
         worksheet.write(cnt, 13, row["배우"])

In [74]:
# 네이버 영화리스트 페이지에서 찾고자 하는 영화를 찾는다.
def movie_search_list(row):
    # 딕셔너리 초기화
    movie_dic = {}
    movie_dic['배급사'] = ""
    movie_dic['등급'] = ""
    movie_dic['장르'] = ""
    movie_dic['배우'] = ""
    movie_dic['개봉일'] = ""
    movie_dic['감독'] = ""
    movie_dic['대표국적'] = ""
    
    movie_name = row['영화명']
    director_name = row["감독"]
    nation = row["대표국적"]
    genre = row['장르']
    
    time.sleep(1)

    # 네이버 영화 페이지에서 영화명 검색
    url = 'https://movie.naver.com/movie/search/result.naver?section=movie&query=' + str(movie_name)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'lxml', from_encoding='utf-8')

    # 검색되는 영화이름 모두 찾기
    # ul 태그의 클래스 search_list_1 찾음
    items = soup.find("ul", {"class": "search_list_1"})

    # 검색되는 영화가 있을 경우 확인
    if (items != None):
        # ul 태그 아래의 li 태그를 모두 찾음
        li_items = items.find_all('li')

        # 결과값을 담을 리스트 생성
        item_list = []
        for li_item in li_items:

            # 결과값을 담을 딕셔너리 생성
            item_dict = {}
            # 영화제목, URL, 감독 값을 태그에서 찾아 딕셔너리
            # 감독명과 타이틀에서 공백을 제거하고 리스트에 넣는다.
            name_tag = li_item.find('dt')
            name = name_tag.text
            if (name.find('(') > -1):
                name = ''.join(name[:name.find('(') - 1].split())  # 모든 공백제거
                item_dict['타이틀'] = name
            else:
                name = ''.join(name.split())  # 모든 공백제거
                item_dict['타이틀'] = name

            movie_url = name_tag.find('a')['href']
            item_dict['주소'] = movie_url
            dd_items = li_item.find_all("dd", {"class": "etc"})
            # 감독 유무

            if director_name == "":
                li_text = str(li_item.text)

                if name_match_ratio(item_dict.get('타이틀'),movie_name) == 100 and li_text.find(nation) > -1 and  li_text.find(genre) > -1:
                    director= None_director_find(dd_items)
                    item_dict['감독'] = director
                    director_name = director
                else:
                    item_dict['감독'] = None
            else:
                director = None_director_find(dd_items) # 네이버 크롤링 데이터
                item_dict['감독'] = director


            # 리스트에 딕셔너리를 넣는다
            item_list.append(item_dict)



        #  네이버에서 가져온 영화정보를 CSV 파일에서 가져온 값과 비교를 해서 정말 그 영화인지 찾는다.
        for naver_item in item_list:

            title = naver_item.get('타이틀')
            director = naver_item.get('감독')
            url = str(naver_item.get('주소')).replace('/movie/bi/mi/basic.naver?code=', '')

            director_name =  ''.join(str(director_name).split()) # 전처리 데이터의 감독명 공백제거
            movie_name = ''.join(str(movie_name).split()) # 전처리 데이터 영화이름 공백제거

            if(director != None ):
                director_ratio = name_match_ratio(director,director_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                # 감독을 찾고 감동의 일치율 넣어준다.
                director_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%

            if(title != None):
                movie_name_ratio = name_match_ratio(title,movie_name) # 크롤링한 감독명과 전처리 데이터의 단어일치율 계산
            else:
                movie_name_ratio = 0 # 크롤링 한 대상에 감독명이 없을 경우 0%


            if (director_ratio >= 60 and movie_name_ratio == 100):

                url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=' + str(url)
                response = requests.get(url)
                soup = BeautifulSoup(response.content, "lxml", from_encoding="utf-8")

                movie_dic['배급사'] = movie_distribution(soup) # 배급사 찾기함수
                movie_dic['등급'] = movie_grade(soup) # 등급찾기 함수
                movie_dic['장르'] = movie_genre(soup) # 장르찾기 함수
                movie_dic['배우'] = movie_actor(soup) # 배우찾기
                movie_dic['개봉일'] = movie_date(soup)
                movie_dic['감독'] = director_name
                movie_dic['대표국적'] = nation

                print(' -- 감독명 : ', director, "||", director_name, '-->  일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title,'-->  일치율: ' ,movie_name_ratio, "%")
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')
            else:

                print(' -- 감독명 : ',director,"||",director_name, '--> 일치율: ' ,director_ratio, "%" )
                print(' -- 영화이름 : ', movie_name, "||", title ,'--> 일치율: ' ,movie_name_ratio, "%" )
                print(' -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-')

    return movie_dic

In [75]:
def None_director_find(dd_items):
    for dd_item in dd_items:
        dd_tag = dd_item.text

        if (dd_tag.find('감독') != -1):
            if (dd_tag.find('|') > -1):
                director_tag = dd_tag[:dd_tag.find('|')]
                director = str(director_tag).replace('감독 : ', "")  # 감독 단어제거
                director = ''.join(director.split())  # 모든 공백제거

            else:
                director = dd_tag.replace('감독 : ', "")
                director = ''.join(director.split())
        else:
            director = None

    return director

In [76]:
# 두 단어간 일치 비율을 리턴한다.
def name_match_ratio(word1,word2):

    d1 = list(filter(None,list(word1))) # 공백문자제거
    d1 = list(set(d1).intersection(d1)) # 중복되는 글자 원소 한개만 남김
    d2 = list(filter(None,list(word2))) # 공백문자제거
    d2 = list(set(d2).intersection(d2)) # 중복되는 글자 원소 한개만 남김
    d3 = list(set(d1).intersection(d2)) # 서로 동일한 단어를 리스트화
    if(word1 == word2):
        ratio = 100 #두단어를 비교했는데 정확하게 일치하면 100%
    else:
        if(len(d1) > 0): # 분모가 0 인경우에는 나눌수 없음.
            # 두단어를 비교했는데 같은 글자의 갯수를 찾고
            ratio = int(len(d3) / len(d1) * 100) # 두 리스트 원소갯수를 비교하여 백분율로 변환
        else:
            ratio = 0

    return ratio

In [77]:
# 배급사정보를 html에서 찾는다.
def movie_distribution(soup):
    distribution = ''

    items = soup.find("dl", {"class": "agency_name"})
    # 배급사 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 배급사 정보 체크
            if (i.text == '배급'):
                distribution = str(dd_items[idx].text).replace('\n', '').replace('\t','').replace('\r','')
    return distribution

In [78]:
# 등급을 html에서 찾는다.
def movie_grade(soup):
    grade = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 등급 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 등급 정보 체크
            if (i.text == '등급'):
                grad_line = str(dd_items[idx].text).replace('\n', '').replace('\r', '').replace('\t', '')
                if(grad_line.find('[해외]')> -1):
                    grade = grad_line[:grad_line.find('[해외]')].replace('[국내]','')
                else:
                    grade = grad_line
    return grade

In [79]:
def movie_genre(soup):
    genre = ''

    items = soup.find("dl", {"class": "info_spec"})
    # 장르 정보가 없을 경우 체크
    if (items != None):

        dt_items = items.find_all('dt')
        dd_items = items.find_all('dd')
        for idx, i in enumerate(dt_items):
            # 장르 정보 체크
            if (str(i.text).find('개요') > -1):
                dd_spans = dd_items[idx].find_all('span')
                genre = str(dd_spans[0].text).replace('\n', '').replace('\r', '').replace('\t', '').split(',')[0]
    return genre

In [80]:
def movie_actor(soup):
    actor = ''
    actor_namelist = []
    items = soup.find("ul", {"class": "lst_people"})
    # 배우 정보가 없을 경우 체크
    if (items != None):
        # 모든 배우이름을 찾는다.
        a_items = items.find_all('a', {"class": "k_name"})
        # 배우이름을 담을 리스트를 만든다.

        # 찾은 모든 배우를 찾아 리스트에 담는다.
        for idx, i in enumerate(a_items):
            # 배우 정보 체크
            actor_name = str(i.text)
            actor_namelist.append(actor_name)
            # 4번째부터는 안넣음.
            if(idx == 2):
                break
    if actor_namelist != None:
        actor = ",".join(actor_namelist)

    return actor

In [81]:
def movie_date(soup):
    date = ''

    info_spec = soup.find("dl", {"class": "info_spec"}) # 개봉일이 들어있는 info_spec 를 찾고
    if(info_spec != None): 
        span_all_tag = info_spec.find_all('span') # span 중에 하나에 개봉일이 들어 있으니 다 찾고
        if(span_all_tag != None): 
            for span_tag in span_all_tag: # 리스트화 되어 있는 span을 반복문에 넣어 하나씩 찾고
                if(span_tag != None): 
                    open_day = str(span_tag.text) # span 태그에서 텍스트만 추출하고

                    if(open_day.find('개봉') > -1): # 개봉이라는 글자를 찾고
                        date = re.sub(r"\s+", "", open_day).replace('.', '-').replace('개봉', '').replace('\n', '').replace('\r', '').replace('\t', '') # 모든 공백을 제거하고

                        if(open_day.find('재개봉') > -1):
                            re_open_day = open_day[:open_day.find('재개봉')]
                            date = open_day[open_day.find('재개봉')+4:].replace('\n', '').replace('\r', '').replace('\t', '').replace('.', '-').replace('개봉', '')

    return date

In [ ]:
if __name__ == '__main__':
    main()

전처리데이터: 1 태극기 휘날리며  ->  {'배급사': '(주)다자인소프트,(주)쇼박스', '등급': '15세관람가', '장르': '전쟁,드라마', '배우': '장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁,이건우,박태원,배상철,김범주,한선랑,이동혁,한재민,지중현,김대영,민지혁,이호천,임동빈,김재중,박성희,임성택,정승진,이정훈,오명준,이기수,김용현,이주원,이환희,장민호,정호빈,박길수,박동빈,안길강,김해곤,권태원,최교식,조운,전승재', '개봉일': '2004-02-05', '감독': '강제규', '대표국적': '한국'}
--------------------------------------------------
전처리데이터: 2 트로이  ->  {'배급사': '워너브러더스 코리아(주)', '등급': '15세관람가', '장르': '사극', '배우': '다이앤 크루거,올랜도 블룸,에릭 바나', '개봉일': '2004-05-21', '감독': '볼프강 페터젠', '대표국적': '미국'}
--------------------------------------------------
전처리데이터: 3 내 머리 속의 지우개  ->  {'배급사': 'CJ ENM', '등급': '12세관람가', '장르': '멜로/로맨스', '배우': '정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,박윤희,이정인,신현탁,김중기,권병길,진용욱,정민성,최교식', '개봉일': '2004-11-05', '감독': '이재한', '대표국적': '한국'}
--------------------------------------------------
전처리데이터: 4 귀신이 산다  ->  {'배급사': '(주)시네마서비스', '등급': '12세관람가', '장르': '코미디,공포(호러),판타지', '배우': '김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양진,이종화,박성웅,장항선,윤문식,오정세,황석정,윤제문,

 -- 감독명 :  프랭크코라치 || 프랑코코라시 --> 일치율:  50 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  마이클앤더슨 || 프랑코코라시 --> 일치율:  0 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  버즈컬릭 || 프랑코코라시 --> 일치율:  0 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  프랭크코라치 || 프랑코코라시 --> 일치율:  50 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  마이클앤더슨 || 프랑코코라시 --> 일치율:  0 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  버즈컬릭 || 프랑코코라시 --> 일치율:  0 %
 -- 영화이름 :  80일간의세계일주 || 80일간의세계일주 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 52 80일간의 세계 일주  ->  {'배급사': '', '등급': '', '장르': '', '배우': '', '개봉일': '', '감독': '', '대표국적': ''}
--------------------------------------------------
전처리데이터: 53 인어 공주  ->  {'배급사': 'CJ ENM', '등급': '전체관람가', '장르': '드라마', '배우': '전도연,박해일,고두심,김봉근,이

 -- 감독명 :  낸시마이어스 || 낸시마이어스 -->  일치율:  100 %
 -- 영화이름 :  사랑할때버려야할아까운것들 || 사랑할때버려야할아까운것들 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  낸시마이어스 || 낸시마이어스 -->  일치율:  100 %
 -- 영화이름 :  사랑할때버려야할아까운것들 || 사랑할때버려야할아까운것들 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 92 사랑할 때 버려야 할 아까운 것들  ->  {'배급사': '워너 브러더스 코리아(주)', '등급': ' 15세 관람가 ', '장르': '코미디', '배우': '잭 니콜슨,다이안 키튼,키아누 리브스', '개봉일': '2004-02-13', '감독': '낸시마이어스', '대표국적': '미국'}
--------------------------------------------------
전처리데이터: 93 킬 빌2  ->  {'배급사': 'CJ ENM', '등급': '18세관람가', '장르': '액션', '배우': '우마 서먼,데이빗 캐러딘', '개봉일': '2004-05-14', '감독': '쿠엔틴 타란티노', '대표국적': '미국'}
--------------------------------------------------
전처리데이터: 94 내셔널 트레져  ->  {'배급사': '한국소니픽쳐스릴리징브에나비스타영화㈜', '등급': '12세관람가', '장르': '액션', '배우': '다이앤 크루거,니콜라스 케이지', '개봉일': '2004-12-31', '감독': '존 터틀타웁', '대표국적': '미국'}
--------------------------------------------------
전처리데이터: 95 택시: 더 멕시멈  ->  {'배급사': '이십세기폭스필름코퍼레이션', '등급': '12세관람가', '장르': '액션', '배우': '지젤 번천,지미

 -- 감독명 :  고수경 || 페드로알모도바르 --> 일치율:  0 %
 -- 영화이름 :  나쁜교육 || 나쁜교육 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  페드로알모도바르 || 페드로알모도바르 -->  일치율:  100 %
 -- 영화이름 :  나쁜교육 || 나쁜교육 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  고수경 || 페드로알모도바르 --> 일치율:  0 %
 -- 영화이름 :  나쁜교육 || 나쁜교육 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  페드로알모도바르 || 페드로알모도바르 -->  일치율:  100 %
 -- 영화이름 :  나쁜교육 || 나쁜교육 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 178 나쁜 교육  ->  {'배급사': '', '등급': '[국내] 청소년 관람불가', '장르': '멜로/로맨스', '배우': '가엘 가르시아 베르날,펠레 마르티네즈,다니엘 기메네즈 카쵸', '개봉일': '  2004-09-17 ', '감독': '페드로알모도바르', '대표국적': '스페인'}
--------------------------------------------------
전처리데이터: 179 춤추는 대수사선 2  ->  {'배급사': '(주)오시네마', '등급': '12세관람가', '장르': '액션,범죄,코미디', '배우': '오다 유지,후카츠 에리', '개봉일': '2003-12-12', '감독': '모토히로 카츠유키', '대표국적': '일본'}
--------------------------------------------------
전처리데이터: 180 아이덴티티  ->  {'배급사': '콜럼비아트라이스타', '등급': '15세관람가', '장르': '범죄,스릴러', '배우': '존 쿠삭,레

 -- 감독명 :  루크그린필드 || 루크그린필드 -->  일치율:  100 %
 -- 영화이름 :  내겐너무아찔한그녀 || 내겐너무아찔한그녀 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 212 내겐 너무 아찔한 그녀  ->  {'배급사': '이십세기폭스코리아(주)', '등급': ' 청소년 관람불가 ', '장르': '멜로/로맨스', '배우': '에밀 허쉬,엘리샤 커스버트,티모시 올리펀트', '개봉일': '2004-10-08', '감독': '루크그린필드', '대표국적': '미국'}
--------------------------------------------------
 -- 감독명 :  곤사토시 || 곤사토시 -->  일치율:  100 %
 -- 영화이름 :  천년여우 || 천년여우 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  이성강 || 곤사토시 --> 일치율:  0 %
 -- 영화이름 :  천년여우 || 천년여우여우비 --> 일치율:  80 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  최환 || 곤사토시 --> 일치율:  0 %
 -- 영화이름 :  천년여우 || 천년여우구미호 --> 일치율:  57 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  곤사토시 || 곤사토시 -->  일치율:  100 %
 -- 영화이름 :  천년여우 || 천년여우 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  이성강 || 곤사토시 --> 일치율:  0 %
 -- 영화이름 :  천년여우 || 천년여우여우비 --> 일치율:  80 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  최환 || 곤사토시 --> 일치율:  0 %
 -- 영화이름 :  천년여우 || 천년여우구미호 --> 일치율:  5

...

 -- 감독명 :  아오야마신지 || 아오야마신지 -->  일치율:  100 %
 -- 영화이름 :  AA || AA -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  제한기르수르티 || 아오야마신지 --> 일치율:  0 %
 -- 영화이름 :  AA || 미래를찍는셔터 --> 일치율:  0 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  실라디트야보라 || 아오야마신지 --> 일치율:  16 %
 -- 영화이름 :  AA || 대디스갓무브스 --> 일치율:  0 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  조신양 || 아오야마신지 --> 일치율:  33 %
 -- 영화이름 :  AA || AA제생활 --> 일치율:  25 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  오카모토기하치 || 아오야마신지 --> 일치율:  14 %
 -- 영화이름 :  AA || 아아폭탄 --> 일치율:  0 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  해빕알라 || 아오야마신지 --> 일치율:  0 %
 -- 영화이름 :  AA || 다그아,샤디 --> 일치율:  0 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  모하나크리쉬나인드라간티 || 아오야마신지 --> 일치율:  0 %
 -- 영화이름 :  AA || AnthakaMunduAaTarvatha --> 일치율:  8 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  니디샤르마 || 아오야마신지 --> 일치율:  20 %
 -- 영화이름 :  AA || 푸자키벤아 --> 일치율:  0 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  None || 아오야마신지 --> 일치율:  0 %
 -- 영화이름 :  AA || 결혼 -->

 -- 감독명 :  얀치트코비치 || 원신연 --> 일치율:  0 %
 -- 영화이름 :  빵과우유 || 빵과우유 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  원신연 || 원신연 -->  일치율:  100 %
 -- 영화이름 :  빵과우유 || 빵과우유 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  얀치트코비치 || 원신연 --> 일치율:  0 %
 -- 영화이름 :  빵과우유 || 빵과우유 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  원신연 || 원신연 -->  일치율:  100 %
 -- 영화이름 :  빵과우유 || 빵과우유 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2057 빵과 우유  ->  {'배급사': '', '등급': '', '장르': '한국', '배우': '원풍연', '개봉일': '', '감독': '원신연', '대표국적': '한국'}
--------------------------------------------------
 -- 감독명 :  마티나스카펠리 || 세미카플라노글루 --> 일치율:  14 %
 -- 영화이름 :  에그 || 에그 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  마리안나팰카 || 세미카플라노글루 --> 일치율:  16 %
 -- 영화이름 :  에그 || 에그 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  구한얼 || 세미카플라노글루 --> 일치율:  0 %
 -- 영화이름 :  에그 || 에그! --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  안쿠스터만즈 || 세미카플라노글루 --> 일치율:  0 %


 -- 감독명 :  하칸베르타스 || 하칸베르타스 -->  일치율:  100 %
 -- 영화이름 :  라다크의아이스하키소녀들 || 라다크의아이스하키소녀들 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  하칸베르타스 || 하칸베르타스 -->  일치율:  100 %
 -- 영화이름 :  라다크의아이스하키소녀들 || 라다크의아이스하키소녀들 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2063 라다크의 아이스하키 소녀들  ->  {'배급사': '', '등급': '', '장르': '다큐멘터리', '배우': '', '개봉일': '', '감독': '하칸베르타스', '대표국적': '스웨덴'}
--------------------------------------------------
 -- 감독명 :  도미니크베셀리 || 도미니크베셀리 -->  일치율:  100 %
 -- 영화이름 :  리버스샷:감독들의반란 || 리버스샷:감독들의반란 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  도미니크베셀리 || 도미니크베셀리 -->  일치율:  100 %
 -- 영화이름 :  리버스샷:감독들의반란 || 리버스샷:감독들의반란 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2064 리버스 샷: 감독들의 반란  ->  {'배급사': '', '등급': '', '장르': '다큐멘터리', '배우': '하크 봄,라이너 베르너 파스빈더,한스 W. 게이센도퍼', '개봉일': '', '감독': '도미니크베셀리', '대표국적': '독일'}
--------------------------------------------------
 -- 감독명 :  박찬경 || 타케후지카요 --> 일치율:  0 %
 -- 영화이름 :  반신반의 || 반신반의 --> 일치율:  100 %
 

 -- 감독명 :  트린T.민하 || 트린민하 -->  일치율:  66 %
 -- 영화이름 :  밤의여로 || 밤의여로 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  None || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 밤의여로 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  김시화 || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 밤으로의긴여로 --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  이재국 || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 여로:기억의밤 --> 일치율:  57 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  시드니루멧 || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 밤으로의긴여로 --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  프란시스레이드,데보라호프만 || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 낮을향한밤의긴여로 --> 일치율:  44 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  조나단밀러 || 트린민하 --> 일치율:  0 %
 -- 영화이름 :  밤의여로 || 밤으로의긴여로 --> 일치율:  66 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2081 밤의 여로  ->  {'배급사': '', '등급': '', '장르': '미국', '배우': '', '개봉일': '', '감독': '트린민하', '대표국적': '미국'}
--------------------------------------------------
크롤링데이터: 2082 비망록: 보이지 않는 도시  ->

 -- 감독명 :  김진열 || 김진열 --> 일치율:  100 %
 -- 영화이름 :  진옥언니학교가다 || 진옥언니,학교가다 --> 일치율:  88 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  김진열 || 김진열 --> 일치율:  100 %
 -- 영화이름 :  진옥언니학교가다 || 진옥언니,학교가다 --> 일치율:  88 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2091 진옥 언니 학교 가다  ->  {'배급사': '', '등급': '', '장르': '', '배우': '', '개봉일': '', '감독': '', '대표국적': ''}
--------------------------------------------------
 -- 감독명 :  이네살림 || 이네살림 -->  일치율:  100 %
 -- 영화이름 :  파리의지붕밑 || 파리의지붕밑 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  르네클레르 || 이네살림 --> 일치율:  25 %
 -- 영화이름 :  파리의지붕밑 || 파리의지붕밑 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  이네살림 || 이네살림 -->  일치율:  100 %
 -- 영화이름 :  파리의지붕밑 || 파리의지붕밑 -->  일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
 -- 감독명 :  르네클레르 || 이네살림 --> 일치율:  25 %
 -- 영화이름 :  파리의지붕밑 || 파리의지붕밑 --> 일치율:  100 %
 -+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-
크롤링데이터: 2092 파리의 지붕 밑  ->  {'배급사': '', '등급': '', '장르': '프랑스', '배우': '미첼 피콜리,밀레느 드몽죠,모리스 베니초우', '개봉일': '', '감독': '이네살림', '대표국적': 